In [100]:
%matplotlib inline
import pandas as pd
import numpy as np

In [101]:
participation = pd.read_csv("data/users_participating_events.csv")

In [102]:
participation.head()

,user_facebook_id,rsvp_status,event_facebook_id
0,201098027006232,attending,1606969096264134
1,334566996897353,attending,1606969096264134
2,354393944896674,attending,1606969096264134
3,1698350500494930,attending,1606969096264134
4,1775177359410467,attending,1606969096264134


In [103]:
events = pd.read_csv("data/events.csv", index_col='facebook_id')
events = events.loc[:, ['attending_count', 'facebook_page', 'name', 'place_name', 'owner_id', 'owner_name']]

In [104]:
participation.head()

,user_facebook_id,rsvp_status,event_facebook_id
0,201098027006232,attending,1606969096264134
1,334566996897353,attending,1606969096264134
2,354393944896674,attending,1606969096264134
3,1698350500494930,attending,1606969096264134
4,1775177359410467,attending,1606969096264134


In [105]:
participation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3667328 entries, 0 to 3667327
Data columns (total 3 columns):
user_facebook_id     int64
rsvp_status          object
event_facebook_id    int64
dtypes: int64(2), object(1)
memory usage: 83.9+ MB


In [106]:
# only attending
attending = participation.loc[lambda df: df.rsvp_status == 'attending', ['user_facebook_id', 'event_facebook_id']]

## attending and maybe
# attending = participation.loc[lambda df: df.rsvp_status.isin(['attending', 'maybe']),
#                              ['user_facebook_id', 'event_facebook_id']]

In [107]:
# data cleaning stuff, as some are only in one (consult `1_coinccidenses_exploration`)
common_indices = set(events.index).intersection(set(attending['event_facebook_id']))
attending = attending.loc[lambda df: df['event_facebook_id'].isin(common_indices)]

In [108]:
attending.head()

,user_facebook_id,event_facebook_id
0,201098027006232,1606969096264134
1,334566996897353,1606969096264134
2,354393944896674,1606969096264134
3,1698350500494930,1606969096264134
4,1775177359410467,1606969096264134


In [109]:
counts = attending.groupby('event_facebook_id').size().to_frame().rename(columns={0: 'count'})

In [110]:
counts.head()

,count
event_facebook_id,
100668907024083,201
100671793684638,23
100755470299906,1
100860333400630,21
100950866765022,28


In [111]:
pairs_raw = attending.merge(attending, on='user_facebook_id')

In [112]:
pairs_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19728557 entries, 0 to 19728556
Data columns (total 3 columns):
user_facebook_id       int64
event_facebook_id_x    int64
event_facebook_id_y    int64
dtypes: int64(3)
memory usage: 602.1 MB


In [113]:
pairs = pairs_raw.groupby(['event_facebook_id_x', 'event_facebook_id_y']).size() \
  .reset_index().rename(columns={0: 'count'})

In [114]:
pairs = pairs \
  .merge(counts, left_on='event_facebook_id_x', right_index=True, suffixes=['', '_x']) \
  .merge(counts, left_on='event_facebook_id_y', right_index=True, suffixes=['', '_y'])

In [115]:
n_users = len(set(attending['user_facebook_id']))
n_users

273335

In [116]:
pairs['pmi'] = np.log(n_users * pairs['count'] / (pairs['count_x'] * pairs['count_y']))

In [117]:
# is it correct? or jsut because matrix is sparse I miss 'zeros'?
pairs['pmi'].describe()

count    5.546890e+06
mean     4.741575e+00
std      1.965660e+00
min     -4.734954e+00
25%      3.511454e+00
50%      4.630869e+00
75%      5.843892e+00
max      1.251845e+01
Name: pmi, dtype: float64

In [119]:
pairs.shape

(5546890, 6)

In [120]:
pairs.loc[lambda df: df['count'] > 1].shape

(2594099, 6)

In [121]:
pairs.loc[lambda df: df['count'] > 2].shape

(1634938, 6)

In [122]:
pairs.head()

,event_facebook_id_x,event_facebook_id_y,count,count_x,count_y,pmi
0,100668907024083,100668907024083,201,201,201,7.215149
2511,100671793684638,100668907024083,5,23,201,5.689092
3381,100860333400630,100668907024083,2,21,201,4.863773
7244,101225040052676,100668907024083,2,35,201,4.352948
9238,101593436871236,100668907024083,3,36,201,4.730242


In [79]:
pairs.to_csv("data/processed/pair_count.csv", index=False)

In [99]:
# pairs.loc[lambda df: df['count'] > 2].to_csv("data/processed/pair_count_maybe_3.csv", index=False)

## To do

* per miejsce
* per organizacja

x

* conditional probability (done, sort of)
* PMI

x

* takie into account attending and interested